# Road Following by Classification - データ収集

## はじめに

画像分類AIを使ってロボットを自律走行させましょう。誰が一番速く走らせることができるかタイムトライアルレースをします。

最初に画像分類AIを学習させるための教師データを収集しましょう。

## 準備

### カメラの確認
カメラの映像がリアルタイムに見えるか確認しましょう。ただしく動かない場合は他のプログラムがカメラを使用していたり，最後にカメラを使ったときに正しく終了されていない可能性があります。

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### Robotの作成
ロボットのモーターを制御するためにRobotクラスのインスタンスを作成しましょう。

In [ ]:
from jetbot import Robot
robot = Robot()

### データディレクトリ
画像データを保存するためのディレクトリを作成しましょう。

In [ ]:
import os

forward_dir = 'dataset/forward'
left_dir = 'dataset/left'
right_dir = 'dataset/right'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(forward_dir)
    os.makedirs(left_dir)
    os.makedirs(right_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


### GUIコントローラ

ロボットを制御するためのGUIコントローラを作成しましょう。

In [ ]:
try:
    del forward_button
    del left_button
    del right_button

except:
    pass

# base button layout
button_layout = widgets.Layout(width='74px', height='50px', align_self='center')

# create counter widgets
forward_count = widgets.IntText(layout=button_layout, value=len(os.listdir(forward_dir)))
left_count = widgets.IntText(layout=button_layout, value=len(os.listdir(left_dir)))
right_count = widgets.IntText(layout=button_layout, value=len(os.listdir(right_dir)))

# create acution button widgets
forward_button = widgets.Button(description='forward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# layout counters and action button widgets
counters = widgets.HBox([left_count, forward_count, right_count])
actions = widgets.HBox([left_button, forward_button, right_button])
counters_box = widgets.VBox([actions, counters], align_self='center')

# display controller box
display(counters_box)

### ロボットの動作関数の定義
左右のモーターの出力値を設定してロボットを動作させるための関数を定義しましょう。

In [ ]:
import time

def move_forward():
    robot.set_motors(0.3, 0.3)
    time.sleep(0.2)
    robot.stop()

def turn_left():
    robot.set_motors(0, 0.3)
    time.sleep(0.2)
    robot.stop()

def turn_right():
    robot.set_motors(0.3, 0)
    time.sleep(0.2)
    robot.stop()

### データ保存関数
画像データを保存するための関数を定義しましょう。

In [ ]:
from uuid import uuid1
from datetime import datetime

def save_snapshot(directory):
    now = datetime.now()
    now = now.strftime("%Y%m%d_%H%M%S")
    image_path = os.path.join(directory, now + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_forward():
    global forward_dir, forward_count
    save_snapshot(forward_dir)
    move_forward()
    forward_count.value = len(os.listdir(forward_dir))

def save_left():
    global left_dir, left_count
    save_snapshot(left_dir)
    turn_left()
    left_count.value = len(os.listdir(left_dir))

def save_right():
    global right_dir, right_count
    save_snapshot(right_dir)
    turn_right()
    right_count.value = len(os.listdir(right_dir))

    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
forward_button.on_click(lambda x: save_forward())
left_button.on_click(lambda x: save_left())
right_button.on_click(lambda x: save_right())

## データ収集

ボタンを押してロボットを動かしながらデータを収集しましょう。

In [ ]:
display(widgets.VBox([image, counters_box]))

## カメラの停止
データ収集が終わったらカメラを止めましょう。これをきちんとやっておかないと他のプログラムでカメラが使用できません。

In [ ]:
camera.stop()

## dataset.zipの作成
`dataset`ディレクトリを`dataset.zip`に圧縮しましょう。これをGoogle Colabに移して画像分類AIを学習させます。

In [ ]:
!zip -r -q dataset.zip dataset


---

## ***(Danger Zone!!) データセットの消去***

次セルのコードを実行すると全てのデータが消去されます。本当に消去する場合はコメントアウト＃を消してから実行してください。

In [ ]:
# import shutil
# shutil.rmtree("dataset")